In [1]:
import geopandas as gpd
import pandas as pd
import os
from glob import glob
from sklearn.linear_model import LinearRegression
from tqdm.auto import tqdm
from tqdm.contrib.concurrent import process_map
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, root_mean_squared_error

In [2]:
cells = gpd.read_file('global-inputs/HR6-change-cells-aoi.gpkg')

cell_ids = cells.hex_id.tolist()

In [3]:
shoreline_files = pd.Series(sorted(glob("data/HR6/*/shoreline_timeseries_tidal_correction.csv")))

In [4]:
def calc_trends(file):
    df  = pd.read_csv(file)
    try:
        df.date = pd.to_datetime(df.date)
    except:
        print(file)
    df.index = (df.date - df.date.min()).dt.days / 365.25
    df.drop(columns=['date', 'Unnamed: 0', 'corrected_IW_shoreline_position', 'EOV_shoreline_chg'], inplace=True, errors='ignore')
    trends = {}
    for col in df.columns:
        sub_df = df[col].dropna()
        
        if not len(sub_df):
            continue
        x = sub_df.index.to_numpy().reshape(-1,1)
        y = sub_df
        linear_trend = LinearRegression().fit(x,y)
        pred = linear_trend.predict(x)

        trends['index'] = file.split('/')[2]
        if col == 'normalised_IW_shoreline_position':
            trends["IW shoreline trend"] = linear_trend.coef_[0]
            trends["IW shoreline intercept"] = linear_trend.intercept_
            trends["IW shoreline n_points"] = len(df[col])
            trends["IW shoreline n_points_nonan"] = len(sub_df)
            trends["IW shoreline r2_score"] = r2_score(y, pred)
            trends["IW shoreline mae"] = mean_absolute_error(y, pred)
            trends["IW shoreline mse"] = mean_squared_error(y, pred)
            trends["IW shoreline rmse"] = root_mean_squared_error(y, pred)
        else:
            trends["EOV shoreline trend"] = linear_trend.coef_[0]
            trends["EOV shoreline intercept"] = linear_trend.intercept_
            trends["EOV shoreline n_points"] = len(df[col])
            trends["EOV shoreline n_points_nonan"] = len(sub_df)
            trends["EOV shoreline r2_score"] = r2_score(y, pred)
            trends["EOV shoreline mae"] = mean_absolute_error(y, pred)
            trends["EOV shoreline mse"] = mean_squared_error(y, pred)
            trends["EOV shoreline rmse"] = root_mean_squared_error(y, pred)

    return pd.DataFrame([trends])
    

In [5]:
trends = pd.concat([calc_trends(f) for f in shoreline_files])

trends.dropna(inplace=True)

trends

/home/ubuntu/miniforge3/envs/coastal-monitoring/lib/python3.13/site-packages/sklearn/metrics/_regression.py:1211: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/home/ubuntu/miniforge3/envs/coastal-monitoring/lib/python3.13/site-packages/sklearn/metrics/_regression.py:1211: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)


/home/ubuntu/miniforge3/envs/coastal-monitoring/lib/python3.13/site-packages/sklearn/metrics/_regression.py:1211: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/home/ubuntu/miniforge3/envs/coastal-monitoring/lib/python3.13/site-packages/sklearn/metrics/_regression.py:1211: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/home/ubuntu/miniforge3/envs/coastal-monitoring/lib/python3.13/site-packages/sklearn/metrics/_regression.py:1211: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)


/home/ubuntu/miniforge3/envs/coastal-monitoring/lib/python3.13/site-packages/sklearn/metrics/_regression.py:1211: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/home/ubuntu/miniforge3/envs/coastal-monitoring/lib/python3.13/site-packages/sklearn/metrics/_regression.py:1211: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)


/home/ubuntu/miniforge3/envs/coastal-monitoring/lib/python3.13/site-packages/sklearn/metrics/_regression.py:1211: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/home/ubuntu/miniforge3/envs/coastal-monitoring/lib/python3.13/site-packages/sklearn/metrics/_regression.py:1211: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)


/home/ubuntu/miniforge3/envs/coastal-monitoring/lib/python3.13/site-packages/sklearn/metrics/_regression.py:1211: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)


/home/ubuntu/miniforge3/envs/coastal-monitoring/lib/python3.13/site-packages/sklearn/metrics/_regression.py:1211: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)


,index,IW shoreline trend,IW shoreline intercept,IW shoreline n_points,IW shoreline n_points_nonan,IW shoreline r2_score,IW shoreline mae,IW shoreline mse,IW shoreline rmse,EOV shoreline trend,EOV shoreline intercept,EOV shoreline n_points,EOV shoreline n_points_nonan,EOV shoreline r2_score,EOV shoreline mae,EOV shoreline mse,EOV shoreline rmse
0,86ba24017ffffff,0.754761,2.777960,157.0,157.0,0.058314,11.894362,190.073993,13.786733,-0.771420,4.229877,157.0,157.0,0.015716,23.809367,770.069567,27.750127
0,86ba24037ffffff,1.072725,-6.667474,159.0,159.0,0.368984,6.853272,65.646662,8.102263,1.599328,-20.920154,159.0,159.0,0.420149,5.748990,117.758257,10.851648
0,86ba24067ffffff,0.498206,14.277762,203.0,203.0,0.012984,10.311452,419.278806,20.476299,0.595446,-16.641978,203.0,203.0,0.008442,22.884779,925.430491,30.420889
0,86ba2414fffffff,0.368255,4.944292,173.0,173.0,0.018754,6.652419,106.484036,10.319110,0.491410,-13.428382,173.0,173.0,0.048605,7.409304,70.935527,8.422323
0,86ba2415fffffff,0.293307,2.322676,286.0,286.0,0.003081,15.147171,758.722732,27.544922,-0.442925,-8.486781,286.0,286.0,0.003635,27.990655,1465.947505,38.287694
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,86dab3707ffffff,0.901469,21.901466,222.0,222.0,0.085477,19.331221,500.422314,22.370121,0.220302,13.243110,222.0,222.0,0.127191,3.332346,19.168801,4.378219
0,86dab370fffffff,0.323892,27.252915,201.0,201.0,0.019027,15.851997,324.287150,18.007975,-0.161150,19.583462,201.0,201.0,0.003950,13.683184,392.607077,19.814315
0,86dab3737ffffff,1.953625,-13.843882,162.0,162.0,0.632647,7.979527,126.080045,11.228537,0.072761,-1.201097,162.0,162.0,0.051363,0.938234,5.562730,2.358544
0,86dab3757ffffff,-0.678629,22.505016,191.0,191.0,0.073018,15.478607,335.515969,18.317095,0.326063,9.973266,191.0,191.0,0.069691,7.525995,81.445040,9.024691


In [6]:
trends_file_path = 'shoreline_trends_by_cell.geojson'
# if os.path.exists(trends_file_path):
#     cell_trends = gpd.read_file('shoreline_trends_by_cell.geojson')
#     cell_trends.to_crs("EPSG:4326", inplace=True)
#     cell_trends.update(trends)
#     cell_trends.to_file(trends_file_path)
# else:
cell_trends = cells.merge(trends, on='index')
cell_trends.to_crs("EPSG:4326", inplace=True)
cell_trends.to_file(trends_file_path)
    
# cell_trends['hex_id'].to_csv("global-inputs/valid_cells.csv")

cell_trends

,index,hex_id,parent_id,sediment_area_ha,geometry,IW shoreline trend,IW shoreline intercept,IW shoreline n_points,IW shoreline n_points_nonan,IW shoreline r2_score,...,IW shoreline mse,IW shoreline rmse,EOV shoreline trend,EOV shoreline intercept,EOV shoreline n_points,EOV shoreline n_points_nonan,EOV shoreline r2_score,EOV shoreline mae,EOV shoreline mse,EOV shoreline rmse
0,86bb5e18fffffff,86bb5e18fffffff,85bb5e1bfffffff,16.76,"POLYGON ((173.94469 -35.12657, 173.94025 -35.0...",0.286229,-4.320310,129.0,129.0,0.026929,...,107.405463,10.363661,1.256841,-22.462887,129.0,129.0,0.064800,21.657042,827.099113,28.759331
1,86bb5e19fffffff,86bb5e19fffffff,85bb5e1bfffffff,20.00,"POLYGON ((174.01618 -35.12027, 174.01171 -35.0...",0.474741,2.817206,77.0,77.0,0.073889,...,59.034851,7.683414,0.526519,-5.437018,77.0,77.0,0.049005,9.889406,112.430338,10.603317
2,86bb58027ffffff,86bb58027ffffff,85bb5803fffffff,97.20,"POLYGON ((172.71095 -34.47891, 172.70704 -34.4...",0.826371,-0.681890,7.0,7.0,0.081622,...,29.049672,5.389775,3.016152,8.967772,7.0,7.0,0.070048,18.404964,456.612787,21.368500
3,86bb72b4fffffff,86bb72b4fffffff,85bb72b7fffffff,14.08,"POLYGON ((173.87492 -39.18614, 173.87027 -39.1...",-0.363870,26.446637,143.0,143.0,0.004920,...,636.475398,25.228464,1.975454,-61.039399,143.0,143.0,0.058542,33.910147,1491.550908,38.620602
4,86bb2bae7ffffff,86bb2bae7ffffff,85bb2baffffffff,44.20,"POLYGON ((175.14401 -40.80153, 175.13871 -40.7...",1.153833,5.297292,88.0,88.0,0.146359,...,413.799888,20.342072,1.066748,-10.256181,88.0,88.0,0.200244,10.071814,242.197839,15.562707
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
631,86da9285fffffff,86da9285fffffff,85da9287fffffff,2.00,"POLYGON ((171.98283 -41.59482, 171.97888 -41.5...",-0.241159,5.911620,247.0,247.0,0.008374,...,427.677245,20.680359,0.082595,10.777429,247.0,247.0,0.070965,1.323148,5.545755,2.354943
632,86da94ab7ffffff,86da94ab7ffffff,85da94abfffffff,50.36,"POLYGON ((172.82222 -43.2102, 172.81779 -43.17...",-0.163705,-10.448493,169.0,169.0,0.005089,...,346.938159,18.626276,0.310463,-2.475856,169.0,169.0,0.183716,4.286880,28.357788,5.325203
633,86da91d8fffffff,86da91d8fffffff,85da91dbfffffff,5.36,"POLYGON ((171.05489 -42.73298, 171.05131 -42.6...",-0.744172,11.424185,222.0,222.0,0.054063,...,432.702328,20.801498,0.093831,-0.356867,222.0,222.0,0.302556,0.804548,0.906301,0.951998
634,86bb299afffffff,86bb299afffffff,85bb299bfffffff,11.68,"POLYGON ((174.2819 -41.84778, 174.27691 -41.81...",2.910599,-41.412632,305.0,305.0,0.293976,...,709.714994,26.640477,-4.796866,25.565687,305.0,305.0,0.218750,41.307503,2866.617632,53.540803
